# Reinforcement Learning for Inverted Pendulum Control

## Introduction

Control theory, is centred around the behaviour of dynamic systems, and how as
engineers we can force it to behave in a certain way. In the context of Mechanical Engineering, these systems could vary from robotics, to automotive systems, to aerospace and HVAC applications. A large portion of Control design is spent modelling the system or "plant". This requires a good grasp of the system's mechanics, understanding all variables that may influence its behaviour. For complex systems, these dynamics are highly non-linear, while exhibiting large degrees of freedom. In such applications, traditional control techniques may struggle to produce reliable and accurate behaviour. Moreover, we may not even be able to model the system, making any form of controller design impossible. 

In these cases, we can make use of Reinforcement Learning (RL) for controlling such complex systems. RL operates in a model-free fashion, meaning we are not required to have an explicit model of a system's behaviour given various inputs. Using RL, we can effectively sidestep all the disadvantages of model-based controller design. RL generally consists of an agent who, through interacting with an environment, can learn to map various situations to actions. Specifically, the environment provides responses to each a agent's action, using these responses and rewards dependent on various states of the agent, we can condition the agent's decisions towards optimal states. In class, the problems we solved required the action and/or the state to be discretizable. However, for continuous systems, this may not be possible or if it is, it raises the question to what resolution we look to discretize the space or actions. The Soft Actor-Critic architecture is a form of RL that looks to solve this problem. 

In this project, we look to investigate RL's applications and performance in control settings. Specifically, we look to investigate the Soft Actor-Critic (SAC) architecture's ability to control a continuous system, consisting of an inverted pendulum on a cart. 


### The Inverted Pendulum





Control Theory is a field of engineering and mathematics that deals with the behavior of dynamic systems. In the context of Mechanical Engineering, dynamic systems are prevalent and can be found in various applications, ranging from robotics and automation to aerospace and automotive systems.



This document explores the implementation of Soft Actor-Critic (SAC), a state-of-the-art reinforcement learning algorithm. SAC is known for its ability to handle continuous action spaces and incorporate entropy regularization, encouraging exploration in addition to exploitation.

The provided code snippets demonstrate key components of the SAC algorithm, including training and testing procedures. We'll break down each section to understand how the agent learns and evaluates its policy in a given environment.

Let's dive into the details of the SAC algorithm implementation and explore how it can be used to train an intelligent agent for decision-making in complex environments.

In [ ]:
from IPython.display import Video

: 